In [185]:
import json
import pennylane as qml
import pennylane.numpy as np

In [186]:
np.random.seed(1967)

In [188]:
def get_matrix_2(params):
    """
    Args:
        - params (array): The four parameters of the model.
        
    Returns:
        - (matrix): The associated matrix to these parameters.
    """
    
    alpha, beta, gamma, phi = params

    # Put your code here #

    RZ_gamma = np.array([[np.exp(complex(0,-gamma/2)), 0], 
                         [0, np.exp(complex(0, gamma/2))]])
    
    RX_beta = np.array([[np.cos(beta/2), complex(0,-np.sin(beta/2))], 
                        [complex(0,-np.sin(beta/2)), np.cos(beta/2)]])
    
    RZ_alpha = np.array([[np.exp(complex(0,-alpha/2)), 0], 
                         [0, np.exp(complex(0, alpha/2))]])
    
    
    the_matrix = np.multiply(np.exp(complex(0, phi)), np.matmul(RZ_gamma, np.matmul(RX_beta, RZ_alpha)))
    
    # Return the matrix
    return the_matrix

In [189]:
params = np.random.rand(4) * np.pi

m1 = get_matrix(params)
m2 = get_matrix_2(params)

print(m1)
print(m2)
print(m1-m2)

[[ 0.572617  -0.24137019j  0.74654194+0.23775056j]
 [ 0.78225008+0.0439888j  -0.44895819-0.42963513j]]
[[ 0.572617  -0.24137019j  0.74654194+0.23775056j]
 [ 0.78225008+0.0439888j  -0.44895819-0.42963513j]]
[[ 1.11022302e-16-5.55111512e-17j -1.11022302e-16-2.77555756e-17j]
 [-1.11022302e-16+0.00000000e+00j -5.55111512e-17+0.00000000e+00j]]


In [210]:
def get_matrix(params):

    alpha, beta, gamma, phi = params
    
    # Calculate the individual rotation matrices
    rz_gamma = np.array([[np.exp(-1j * gamma / 2), 0], [0, np.exp(1j * gamma / 2)]])

    rx_beta = np.array([[np.cos(beta / 2), -1j * np.sin(beta / 2)],
                        [-1j * np.sin(beta / 2), np.cos(beta / 2)]])
    
    rz_alpha = np.array([[np.exp(-1j * alpha / 2), 0], [0, np.exp(1j * alpha / 2)]])

    # Combine the rotation matrices in the specified order
    unitary_matrix = np.exp(1j * phi) * rz_gamma @ rx_beta @ rz_alpha
    
    return unitary_matrix

In [229]:
def error(U, params):
    """
    This function determines the similarity between your generated matrix and
    the target unitary.

    Args:
        - U (np.array): Goal matrix that we want to approach.
        - params (array): The four parameters of the model.

    Returns:
        - (float): Error associated with the quality of the solution.
    """

    matrix = get_matrix(params)._value
    #print(matrix)
    
    # Put your code here #
    
    error_value = float(np.absolute(np.linalg.norm(U - matrix)))
    
    error_value_2 = float(np.absolute(np.sqrt(np.array(sum([x**2 for x in (matrix - U).flatten()])))))
    
    # Frobenius Inner Product
    error_value_3 = float(np.absolute(sum(np.multiply(np.conjugate(matrix).flatten(), np.array(U).flatten()))))
    print(error_value_3)
    
    #print("ev type:", type(error_value), type(error_value_2), type(error_value_3))
    #print("ev:", error_value, error_value_2, error_value_3)
    
    # Return the error
    return error_value_3


def train_parameters(U):
    epochs = 5
    lr = 0.01

    grad = qml.grad(error, argnum=1)
    params = np.random.rand(4) * np.pi
#    print("params orig:", params)
    
    for epoch in range(epochs):
        params -= lr * grad(U, params)
#        print("params:", params)
        
    return params


In [230]:
# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
    matrix = json.loads(test_case_input)
    params = [float(p) for p in train_parameters(matrix)]
    #print("run params:", params)
    return json.dumps(params)


def check(solution_output: str, expected_output: str) -> None:
    matrix1 = get_matrix(json.loads(solution_output))
    matrix2 = json.loads(expected_output)
    print("matrix1:", matrix1)
    print("matrix2:", matrix2)
    assert not np.allclose(get_matrix(np.random.rand(4)), get_matrix(np.random.rand(4)))
    assert np.allclose(matrix1, matrix2, atol=0.2)


# These are the public test cases
test_cases = [
    ('[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]', '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'),
('[[ 1,  0], [ 0, -1]]', '[[ 1,  0], [ 0, -1]]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[ 0.70710678,  0.70710678], [ 0.70710678, -0.70710678]]'...
1.0636376128760359
1.0636376128760359
1.0636376128760359
1.0636376128760359
1.0636376128760359
matrix1: [[ 0.72111022+0.69104597j -0.0332585 +0.03673402j]
 [ 0.035819  +0.03424201j  0.66944076-0.74121087j]]
matrix2: [[0.70710678, 0.70710678], [0.70710678, -0.70710678]]


AssertionError: 